In [3]:
from pyspark import SparkConf
from pyspark import SparkContext

conf = SparkConf()
sc = SparkContext(conf=conf)

In [95]:
#A vasarlasok.txt egy kiskereskedelmi áruházban megvásárolt termékeket tartalmazzák.
#Minden sor egy vásárló kosarában lévő termékeket sorolja fel, vesszővel elválasztva.
#Határozd meg azt a három terméket, amelyekből a legtöbbet vásárolták és add meg hányat.
#Figyelj oda arra, hogy az adathalmazban kis és nagy betűk is előfordulhatnak.
#Azaz előfordulhat a "rizs" és a "Rizs" termék is. Ezeket azonos termékeknek kell tekintened.

lines = sc.textFile("vasarlasok.txt")\
.flatMap(lambda x : x.split(","))\
.map(lambda word: (word.lower(), 1))\
.reduceByKey(lambda a, b: a + b)\
.sortBy(lambda x: x[1], False)

lines.take(3)

[('kenyer', 108), ('cukor', 68), ('tej', 64)]

In [93]:
#A weboldalak.txt weboldalakat és azok szöveges tartalmát tartalmazza.
#Minden sor egy weboldal címével kezdődik, aztán pedig a tartalma következik.
#A felhasználónk az "ELTE" kifejezésre keres rá, nekünk pedig vissza kell adnunk számára a legrelevánsabb találatot.
#Határozod meg, hogy a weboldalak.txt-ben található weboldalak közül melyikben szerepel legtöbbször a keresett kifejezés.
#Csak azokat a weboldalakat vedd figyelembe, amelyek tartalma több, mint 10 szóból áll.
#A programod írja ki azt a weboldalt, amelyekikben a legtöbbször szerepel a keresett kifejezés.
#Add meg továbbá a keresett kifejezés előfordulásainak a számát és a weboldal tartalmában lévő szavak számát is.

def count(line):
    wordcount = 0
    url = line[0]
    if len(line) > 10:
        for word in line:
            if "ELTE" in word:
                wordcount = wordcount + 1

    return (url, wordcount, len(line))

text = sc.textFile("weboldalak.txt")\
.map(lambda x : x.split(" "))\
.map(count)\
.sortBy(lambda x: x[1], False)

text.take(1)

[('elte.hu', 5, 54)]

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

In [6]:
books = spark.read.format("csv")\
.option('header', True)\
.option("inferschema", True)\
.load("books.csv")

orders = spark.read.format("csv")\
.option('header', True)\
.option("inferschema", True)\
.load("orders.csv")

books.createOrReplaceTempView("booksTable")
orders.createOrReplaceTempView("ordersTable")

In [13]:
#Add meg azt a három szerzőt (author), akik a legtöbb könyvet írták (a null nem számít szerzőnek).

books.filter(col("author").isNotNull()).groupBy("author").count().orderBy(col("count").desc()).show(3)

+--------------------+-----+
|              author|count|
+--------------------+-----+
|     Garcia Santiago| 1479|
|Shelley Admont, K...|  228|
|       James Manning|  180|
+--------------------+-----+
only showing top 3 rows



In [19]:
#Add meg, hogy a felhasználók munkameneteiben (sessionID) átlagosan hány kattintás történik.

orders.groupBy("sessionID").sum("click").select(avg(col("sum(click)"))).show()

+-----------------+
|  avg(sum(click))|
+-----------------+
|1.655570384913763|
+-----------------+



In [33]:
#Melyik könyvből vásárolták meg a legtöbbet? Add meg a könyv címét, és a megvásárolt könyvek darabszámát.

orders.alias('o').join(books.alias('b'), col('o.itemID') == col('b.itemID'), 'inner')\
.groupBy("title").sum("order").orderBy(col("sum(order)").desc()).show(1)

+--------------------+----------+
|               title|sum(order)|
+--------------------+----------+
|Rico, Oskar und d...|       170|
+--------------------+----------+
only showing top 1 row



In [7]:
books.show()

+-----+------+--------------------+------------------+--------------------+
|rowID|itemID|               title|            author|           publisher|
+-----+------+--------------------+------------------+--------------------+
|    0| 21310|Princess Poppy: T...|Janey Louise Jones|Penguin Random Ho...|
|    1| 73018|Einfach zeichnen!...|     Wiebke Krabbe|Schwager und Stei...|
|    2| 19194|         Red Queen 1|  Victoria Aveyard|Orion Publishing ...|
|    3| 40250|Meine Kindergarte...|              null|    Ars Edition GmbH|
|    4| 46107|Mein großes Schab...| Elizabeth Golding|Edition Michael F...|
|    5| 34217|        Ewig geliebt|        J. R. Ward|   Heyne Taschenbuch|
|    6| 31436| Meine Sticker-Tiere|              null|    Ars Edition GmbH|
|    7| 14576|Unsterblich 01 - ...|      Julie Kagawa|   Heyne Taschenbuch|
|    8| 17731|Unsterblich 02 - ...|      Julie Kagawa|   Heyne Taschenbuch|
|    9| 58723|Pedro und die Bet...|     Ursula Hasler|dtv Verlagsgesell...|
|   10| 7312

In [8]:
orders.show()

+-----+---------+------+-----+------+-----+
|rowID|sessionID|itemID|click|basket|order|
+-----+---------+------+-----+------+-----+
|    0|        0| 21310|    1|     0|    0|
|    1|        1| 73018|    1|     0|    0|
|    2|        2| 19194|    1|     0|    0|
|    3|        3| 40250|    1|     0|    0|
|    4|        4| 46107|    1|     0|    0|
|    5|        5| 34217|    1|     0|    0|
|    6|        6| 31436|    1|     0|    0|
|    7|        7| 14576|    1|     1|    0|
|    8|        7| 17731|    2|     1|    0|
|    9|        8| 58723|    1|     0|    0|
|   10|        9| 73124|    1|     0|    0|
|   11|       10| 33976|    1|     0|    0|
|   12|       11|  2408|    2|     0|    0|
|   13|       12| 30277|    1|     0|    0|
|   14|       12| 29508|    1|     1|    0|
|   15|       12| 75659|    1|     0|    0|
|   16|       13| 55699|    2|     0|    0|
|   17|       13| 62433|    1|     0|    0|
|   18|       13| 72603|    1|     0|    0|
|   19|       14| 11989|    1|  